## 16S analysis for seasonal comparison of howler metabolites and microbiome

The following is code used for an analysis of howler metabolites across seasons. Previously published 16S rRNA gene sequencing data was used to contextualize the metabolome data and was reanalyzed for this analysis using more recent methods.

### Converting 454 sequencing files to fastq files
Sequencing was performed on the 454 platform, so we converted the .fna and .qual files to .fastq files in QIIME1.

In [ ]:
source /macqiime/configs/bash_profile.txt

cat file-list.txt | while read i; do 
convert_fastaqual_fastq.py -f ${i}.fasta -q ${i}.fasta.qual -o ./Seqs_fastq
done

### Importing sequences
Fastq files were then imported into QIIME2 (v.2019-4) using the manifest import option.

In [ ]:
source activate qiime2-2019.4

qiime tools import --type 'SampleData[SequencesWithQuality]' --input-path ./Manifest.csv \
    --output-path howlers-demux.qza --input-format SingleEndFastqManifestPhred33V2

qiime demux summarize --i-data howlers-demux.qza --o-visualization howlers-demux.qzv

### Denoising
The DADA2 algorithm was used to denoise sequences, using the 454 parameters suggested by QIIME2. Truncation length was set to 250bp, trim length was set to 10bp, and the max errors allowed were 3.

In [ ]:
qiime dada2 denoise-pyro --i-demultiplexed-seqs ./howlers-demux.qza --p-trunc-len 250 --p-trim-left 10 \
    --p-max-ee 3 --p-n-threads 8 --o-table ./table.qza --o-representative-sequences ./rep-seqs.qza \
    --o-denoising-stats ./dada2-stats.qza
    
qiime metadata tabulate --m-input-file dada2-stats.qza --o-visualization dada2-stats.qzv

qiime feature-table summarize --i-table table.qza --o-visualization table.qzv --m-sample-metadata-file howler_map.txt

### Phylogenetic tree
A phylogenetic tree was constructed from the representative sequences uses QIIME2's MAFFT/FastTree workflow.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree --i-sequences rep-seqs.qza --o-alignment aligned-rep-seqs.qza \
    --o-masked-alignment masked-aligned-rep-seqs.qza --o-tree unrooted-tree.qza --o-rooted-tree rooted-tree.qza

### Taxonomic assignment
Taxonomy was assigned using a naive Bayesian classifier trained on the GreenGenes 13_8 database.

In [ ]:
qiime feature-classifier classify-sklearn --i-classifier ./gg-13-8-99-nb-classifier-qiime2019-4.qza \
    --i-reads ./rep-seqs.qza --o-classification ./taxonomy.qza

### Feature table filtering, collapsing, and exporting
Mitochondrial and chloroplast sequences were filtered out of the feature table. Taxa bar plots were constructed to visualized the data. Feature tables were collapsed at the level of phyla, families, genera, and ASVs and exported.

In [ ]:
qiime taxa filter-table --i-table table.qza --i-taxonomy taxonomy.qza --p-exclude mitochondria,chloroplast \
    --o-filtered-table table-nomito-nochloro.qza

qiime feature-table summarize --i-table table-nomito-nochloro.qza --o-visualization table-nomito-nochloro.qzv \
    --m-sample-metadata-file howler_map.txt
    
qiime taxa barplot --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --m-metadata-file howler_map.txt \
    --o-visualization taxa-plots-nomito-nochloro.qzv

qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 2 \
    --o-collapsed-table table-filtered-level2.qza

qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 5 \
    --o-collapsed-table table-filtered-level5.qza

qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 6 \
    --o-collapsed-table table-filtered-level6.qza

qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 7 \
    --o-collapsed-table table-filtered-level7.qza

qiime tools export --input-path table-filtered-level2.qza --output-path ./output-tables

qiime tools export --input-path table-filtered-level5.qza --output-path ./output-tables

qiime tools export --input-path table-filtered-level6.qza --output-path ./output-tables

qiime tools export --input-path table-filtered-level7.qza --output-path ./output-tables

qiime tools export --input-path table-nomito-nochloro.qza --output-path ./output-tables
qiime tools export --input-path taxonomy.qza --output-path ./output-tables #edit taxonomy header

biom add-metadata -i ./output-tables/feature-table.biom -o ./output-tables/table-with-taxonomy.biom \
    --observation-metadata-fp ./output-tables/biom-taxonomy.tsv --sc-separated taxonomy
    
biom convert -i ./output-tables/table-with-taxonomy.biom -o ./output-tables/table-with-taxonomy.tsv \
    --to-tsv --header-key taxonomy

### Diversity analyses
QIIME2 was used to perform core diversity analyses. Samples were rarefied to 2452 ASVs prior to calculating diversity indices.

In [ ]:
qiime diversity core-metrics-phylogenetic --i-phylogeny rooted-tree.qza --i-table table-nomito-nochloro.qza \
    --p-sampling-depth 2452 --m-metadata-file howler_map.txt --output-dir core-metrics-2452